In [16]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            15Gi        10Gi       277Mi       451Mi       4.9Gi       4.4Gi
Swap:          976Mi       181Mi       795Mi


# PROBLEM STATEMENT & MOTIVATION
-----------------
-----------------

&nbsp;

Brain-Computer Interfaces (BCIs), can be extremely empowering for people with disabilities who choose to use them. There are a few commercially available EEG headsets, such as the Emotiv EPOC (pictured below) that provide less cost prohibitive, noninvasive options to convert electrical signals from the brain into computer commands that might be otherwise inaccessible to input for someone.

&nbsp;

![imageof the Emotiv EPOC+ headset on a white background, next to a schematic of the 10-20 electrode placement system](https://d2z0k1elb7rxgj.cloudfront.net/uploads/2018/11/a-Emotiv-EPOC-headset-b-Spatial-mapping-of-the-electrodes-on-the-scalp.jpg)


&nbsp;


This project aims to use some techniques from the field of data science to explore the feasibility of classiying EEG signals captured by a low cost, dry electrode system such as the Emotiv EPOC+. The data used was collected over nearly two years, 2014-15 and is [curated and hosted by the subject of the readings, David Vivancos](http://mindbigdata.com/opendb/index.html). Although four different datasets using four different devices are available, for this project I decided to analyze the one with the most channels, especially since it was the only one with electrode channels on the occipital lobe, which is the "visual cortex." Lower cost options are available, such using a development board or microcontroller (\\$5 - \\$50) with an amplifier such as [this one](https://biosignals.berndporr.me.uk/#build_your_own_bio-amplifier), with electrodes from any supplier, which can be only [a few dollars](https://www.alibaba.com/product-detail/Colorful-Reusable-Gold-Cup-Electrodes-Cable_1600592681920.html)

In [3]:
# Import the necessary packages

import pandas as pd
import numpy as np
import time
import gc
import scipy
from scipy.signal import hilbert, savgol_filter, wavelets, periodogram
# from sklearn.decomposition import FastICA
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Conv2D, GlobalMaxPooling2D, MaxPooling2D, Flatten
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.svm import SVC

## DATA PROCESSING
-----------------
-----------------

Uncomment and run the cell below to load and process the data for the model.

In [ ]:
# # Load in original raw data and give it column names
# cols = ['id', 'event', 'device', 'channel', 'code', 'size', 'data']
# emotiv = pd.read_csv('../../fulldata/EP1.txt', delimiter='\t', names=cols)
# emotiv.drop(['device', 'id'], inplace=True, axis=1)

# # Break up full df into sub-dfs by channel
# occO1 = emotiv[(emotiv['channel'] == 'O1')]
# occO2 = emotiv[(emotiv['channel'] == 'O2')]
# fefF3 = emotiv[(emotiv['channel'] == 'F3')]
# fefF4 = emotiv[(emotiv['channel'] == 'F4')]
# fefF7 = emotiv[(emotiv['channel'] == 'F7')]
# fefF8 = emotiv[(emotiv['channel'] == 'F8')]
# temT7 = emotiv[(emotiv['channel'] == 'T7')]
# temT8 = emotiv[(emotiv['channel'] == 'T8')]
# pfcAF3 = emotiv[(emotiv['channel'] == 'AF3')]
# pfcAF4 = emotiv[(emotiv['channel'] == 'AF4')]
# motFC5 = emotiv[(emotiv['channel'] == 'FC5')]
# motFC6 = emotiv[(emotiv['channel'] == 'FC6')]
# parP7 = emotiv[(emotiv['channel'] == 'P7')]
# parP8 = emotiv[(emotiv['channel'] == 'P8')]

# # Delete and garbage collect the full df so computer doesn't run out of RAM and freeze
# del emotiv
# gc.collect()

# def dataProcessor(df):
#     '''
# Cleans data column by splitting it into smaller strings, converting those to float, cutting it down to length defined by shortest data vector, normalizing the indexes by resetting.

# i: Dataframe for single channel
# o: Processed dataframe, printouts of lengths before and after clipping for check, timestamp for each iteration
#     '''
    
#     col = df['data'].apply(lambda x: list(map(float, x.split(','))))
#     print(type(col), type(col.iloc[0]), type(col.iloc[0][0]))

#     for i in range(len(col)):
#         l = []
#         l.append(len(col.iloc[i]))

#     print(min(l))

#     for i in range(len(col)):
#         col.iloc[i] = col.iloc[i][:256] # or 257?

#     for i in range(len(col)):
#         l = []
#         l.append(len(col.iloc[i]))

#     print(max(l))
#     return col.reset_index(drop=True)

# # Choose  which channels to include
# dfs = [occ0, occ1, fefF3,
#        fefF4, fefF7, fefF8, temT7,temT8,
#         pfcAF3, pfcAF4, motFC5, motFC6,
#         parP7, parP8]

# # Init blank dataframe for processed channels to be added to
# df = pd.DataFrame()

# #  select columns by name by grabbing channel name value string from the 'channel' column
# # then running dataProcessor on each/any channel dataframes
# for x in dfs:
#     name = x['channel'].iloc[0]
#     df[name] = dataProcessor(x) 
#     print(time.time())

# # Add code column from any channel df
# df['code'] = occ1['code'].reset_index(drop=True)
# print(df.head())
# print(type(df), type(df.iloc[0]), type(df.iloc[0][0]))

# # Delete original dfs with this ugly stack of dels, garbage collect to conserve RAM
# del occ0
# del occ1
# del fefF3
# del fefF4
# del fefF7
# del fefF8
# del temT7
# del temT8
# del pfcAF3
# del pfcAF4
# del motFC5
# del motFC6
# del parP7
# del parP8
# gc.collect()

# # Save resulting dataframe to csv
# df.to_csv('../data/df.csv', sep=';', quoting=None)

In [5]:
#Loading dataset if saved before
df = pd.read_csv('./data/dfTensor.csv', delimiter=';', encoding='latin-1') # This line might show an older version's filename, dfTensor
df.drop('Unnamed: 0', inplace=True, axis=1)

#Check arbitrary column to see if the processing step encoded the data vectors as strings, which it probably did:
df['F8'].iloc[0], df.columns

'[3989.23077, 3983.589744, 3987.692308, 3991.794872, 3992.307693, 3990.256411, 3993.333334, 4000.0, 3994.871795, 3985.128206, 3995.897436, 4001.538461, 3986.153847, 3982.051283, 3990.256411, 3994.358975, 3981.538462, 3961.025642, 3968.205129, 3989.23077, 3980.512821, 3973.333334, 3997.948718, 4009.230769, 3998.974359, 3998.461539, 3999.48718, 3997.435898, 3999.48718, 4001.025641, 3994.871795, 3987.179488, 3987.692308, 3992.307693, 3987.692308, 3980.0, 3984.102565, 3994.358975, 3991.794872, 3982.564103, 3980.0, 3978.461539, 3978.461539, 3989.23077, 3998.461539, 3998.461539, 3996.410257, 3978.974359, 3966.153847, 3978.974359, 3987.692308, 3985.128206, 3993.846154, 3995.384616, 3974.871795, 3957.948718, 3965.128206, 3990.769231, 4007.179487, 3998.974359, 3981.025642, 3970.256411, 3976.923077, 3993.333334, 3992.820513, 3978.461539, 3976.923077, 3975.897436, 3973.846154, 3984.615385, 3986.153847, 3978.974359, 3980.0, 3982.051283, 3981.538462, 3983.589744, 3978.974359, 3971.282052, 3972.8205

In [ ]:
# If modeling a single channel, run this cell

dfO1 = df['O1'] #.explode?
dfO1.applymap(eval)
dfO1.explode # or to numpy array flatten back to df?
# ...

# DATA SETUP
-----------------
-----------------

In [ ]:
# Setting up X and y
X = df.drop('code', axis=1)
y = df['code']

In [ ]:
##Run this cell to convert the values to numerical type before signal processing.
# X = X.applymap(eval) # takes a while
# X = X.applymap(np.array) # try this, maybe at another place, to hopefully fix the shape issue
# print(X.iloc[0], type(y.iloc[0]), X.shape, y.shape)

In [ ]:
# train-test split for "df"
XTrain, XTest, yTrain, yTest = train_test_split(X, y)

In [ ]:
XTrain.shape, XTest.shape, yTrain.shape, yTest.shape

In [ ]:
XTrainNP = XTrain.applymap(np.array)
XTrainNP = XTrainNP.to_numpy()
XTrainNP.shape
XTrainNP = np.reshape(XTrainNP, newshape=(256, 1, 14))

# SIGNAL PREPROCESSING
-----------------
-----------------

## Savitzky-Golay Filter

![Gif of how the filter smoothly approximates the curve at discrete time steps](https://upload.wikimedia.org/wikipedia/commons/8/89/Lissage_sg3_anim.gif)

"The idea of Savitzky-Golay filters is simple – for each sample in the filtered sequence, take its direct neighborhood of N neighbors and fit a polynomial to it. Then just evaluate the polynomial at its center (and the center of the neighborhood), point 0, and continue with the next neighborhood. "


-- https://bartwronski.com/2021/11/03/study-of-smoothing-filters-savitzky-golay-filters/

In [ ]:
XSGF = X[X[['O1', 'O2', 'F3', 'F4', 'F7', 'F8', 'T7', 'T8', 'AF3', 'AF4', 'FC5',
       'FC6', 'P7', 'P8']]].applymap(savgol_filter( 10001, 1 ) )
#dfSGF.concatenate(df['code'])

## Fast ICA

![Logo of the name of the algorithm from its project website from the University of Aalto, Finland ](https://research.ics.aalto.fi/ica/fastica/FastICA.gif)

"Independent component analysis (ICA) is a statistical and computational technique for revealing hidden factors that underlie sets of random variables, measurements, or signals.

ICA defines a generative model for the observed multivariate data, which is typically given as a large database of samples. In the model, the data variables are assumed to be linear mixtures of some unknown latent variables, and the mixing system is also unknown. The latent variables are assumed nongaussian and mutually independent, and they are called the independent components of the observed data. These independent components, also called sources or factors, can be found by ICA. "


-- https://www.cs.helsinki.fi/u/ahyvarin/whatisica.shtml

In [ ]:
fICA = FastICA(5, whiten=True)
XfICA = fICA.fit_transform(X.to_numpy().reshape(-1, 1))

# MODELING
-----------------
-----------------

In [ ]:
# Initiate and setup the model

model = Sequential()
model.add(Conv2D(filters=64, kernel_size=1, activation='relu', input_shape=(256,14,1)))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Conv2D(filters=64, kernel_size=1, activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Conv2D(filters=64, kernel_size=1, activation='relu'))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(11, activation='softmax'))

# Print summary
model.summary()

Most recent model summary:

```
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 conv2d (Conv2D)             (None, 256, 14, 64)       128       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 256, 14, 64)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 14, 64)       4160      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 256, 14, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 256, 14, 64)       4160      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 256, 14, 64)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 229376)            0         
                                                                 
 dense (Dense)               (None, 32)                7340064   
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 11)                363       
                                                                 
=================================================================
Total params: 7,349,931
Trainable params: 7,349,931
Non-trainable params: 0
```

In [ ]:
# Run compile and fit model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
results = model.fit(XTrain, yTrainOHE, batch_size=20,epochs=30, verbose=1)

In [ ]:
# Create a quck Support Vector Classification to check if there is similar performance on a simpler model
svc = SVC()
svc.fit(X, y)

svc.score(XTest, yTest)

## RESULTS
---------------
---------------

The first iterations of the model were using data that was averaged per event. This effectively erased the time data, which is apparently crucial for neural coding (see this [video lecture by Earl Miller for more information on why](https://www.youtube.com/watch?v=Kqyhr9fTUjs)). This seems like it would be made worse by raw data being a measurement of changes in amlitude over time, even with filters. It might perform better the frequency information from Fourier transformation, but mean freqency without time would still lose a lot of information, other than perhaps the dominant frequency band of the channel. 


>
>Best accuracy for the unprocessed signal with all channels was about .1020
> &nbsp;
>
>In 6/14 channels, adding SGF data was about the same
>
> &nbsp;
>
>In 6/14 channels, adding SGF and dropping raw data resulted in really low accuracy and NaN loss
>
> &nbsp;
>
>In 6/14 channels, ICA only had a accuracy of about .1000
>
> &nbsp;
>
>6/14 channels, unproccessed+SGF+ICA, 22,059 params: NaN loss again, what's wrong with the data? woe be the futility of mine ways
>
> &nbsp;
>
>6/14 channels, SGF only, 9,771 params, accuracy still around .1000
>
> &nbsp;
>
>6/14 channels, raw, 9,771 params, accuracy about .1000, so the .0020 increase was only from the extra channels. Processing doesn't seem to change the accuracy at all.
>
> &nbsp;
>
 ----------

 &nbsp;


Realizing the temporal coding is crucial, I ran the model using vector data rows ("wide"). The first iterations of this process indicated that the TF CNN algorithm cannot run on matrices whos values are both scalars and vectors. To correct this, I need to find a way to project it to a lower dimension while preserving the order of the readings. This is currently in process

## BIBLIOGRAPHY
---------------
---------------

&nbsp;

- "Deep learning for electroencephalogram (EEG) classification tasks: a review", Alexander Craik et al 2019 J. Neural Eng. 16 031001, doi:10.1088/1741-2552/ab0ab5

&nbsp;

- ""